In [61]:
import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
from ase.visualize import view
from ase.io import read,write
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
from make_input.utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle,isInSP
from make_input.qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput
from make_input.custom_frame import frame2qe_format,get_ibrav2_frame,get_ibrav9_frame,get_standard_frame
from make_input.qe_run import run_qe_local
from make_input.raw_info import SG2BravaisLattice,missClassificationCorrection
from read_output.parser import extract_floats,finishedProperly,fn2info,fn2num,get_filenames,get_patterns,get_energy_per_atom


# import info and input structures

In [2]:
fileNames = {}
infoPath = './info/'
structurePath = './structures/'
# fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['crystals'] = structurePath + 'input_crystals_sg1-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [3]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

# examples

In [4]:
from make_input.qe_input import frame2change,ibrav0,NOPROBLEM,dont_print_wyck,tricky_sg

## single sg

In [22]:
sg,it = 210,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)


In [23]:
print cc.get_scaled_positions()[0]

[ 0.30321532  0.3032168   0.58934866]


In [24]:
input_str = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                scf_conv_thr=1e-1,print_forces=True,print_stress=False,
                restart=False,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"')
# print inpt

In [ ]:
exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

## several sg

In [19]:
zatom = 14
calculation_type = '"scf"'
sgs = [202]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"./pseudo/SSSP_acc_PBE/"'

dataPath = './test_run/check_input_errors_ibrav3/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [20]:
for (sg,it),dirName in dirNames.iteritems():
    print dirName
    crystal = crystals[sg][it]
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

./test_run/check_input_errors_ibrav3/sg_202-f_0
sending the calc
(0, 'No error')


# ibrav 3 test

In [38]:
sg,it = 220,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)

In [29]:
print cc.get_scaled_positions()[0]

[ 0.62501026  0.76843214  0.48157344]


In [33]:
fns = get_filenames('./test_run/check_input_errors_ibrav3/', fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out',
 './test_run/check_input_errors_ibrav3/sg_202-f_0-1/qe.out']

In [45]:
fn = './test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)

# ibrav 9 test

## preliminary

In [54]:
dataPath = './test_run/check_input_errors_ibrav9/'

In [55]:
sg,it = 40,0
cc = crystals[sg][it]
dirName = dataPath+'sg_{}-f_{}/'.format(sg,it)

In [60]:
view(cc)

In [56]:
print cc.get_cell_lengths_and_angles()
print cc.get_scaled_positions()[0]

[  8.10960373   3.90162918   2.25168312  90.          90.          90.        ]
[ 0.61485391  0.83332859  0.49      ]


In [57]:
fns = get_filenames(dataPath, fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out']

In [59]:
fn = './test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)

## full

In [383]:
zatom = 14
calculation_type = '"scf"'
sgs = [20, 21, 35, 36, 37, 38, 39, 40, 41, 63, 64, 65, 66, 67, 68]
sgs = [35, 65, 68]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav9/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [385]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    data = spg.get_symmetry_dataset(crystal,symprec=symprec)
    
    if len(np.unique(data['equivalent_atoms'])) > 1:
        print '########'
        print sg,it
        print '########'
        continue
    elif sg == 35 and data['wyckoffs'][0] not in ['p','q']:
        continue
    
    print dirName,'  ',data['wyckoffs'][0]
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_2    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_1    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0    c
sending the calc
(0, 'No error')
########
8 5
########
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_4    d
sending the calc
(0, 'No error')


In [134]:
missClassificationCorrection[35]

65

In [431]:
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_8*')
fns

['/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_4-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_3-0/qe.out']

In [432]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [433]:
failed = {}
for fn,(sg,f) in zip(fns,ppp):
    if not finishedProperly(fn):
        with open(fn,'r') as fout:
            failed[(sg,f,fn)] = fout.read()
print len(failed)

0


In [440]:
sg,it = 8,0
cc = crystals[sg][it]
dd = get_standard_frame(cc,sg,primitive=False)
view(dd)
print dd.get_cell()
print dd.get_scaled_positions()
fn = fnd[(sg,it)]
print fn
rr = read(fn)
ss = get_standard_frame(rr,sg,primitive=False)
view(ss)
print ss.get_cell()
print ss.get_scaled_positions()

[[ 3.91330814  0.          0.        ]
 [ 0.          2.26301179  0.        ]
 [-1.30763156  0.          1.84589388]]
[[ -2.83704473e-17   0.00000000e+00   5.00000000e-01]
 [  5.00000000e-01   5.00000000e-01   5.00000000e-01]]
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0-0/qe.out
[[ 3.91330635  0.          0.        ]
 [ 0.          2.26300958  0.        ]
 [-1.30763147  0.          1.84589338]]
[[ 0.5  0.5  0. ]
 [ 0.   0.   0. ]]


In [169]:
dd.get_scaled_positions()

array([[ 0.5       ,  0.74998098,  0.        ],
       [ 0.5       ,  0.25001902,  0.        ],
       [ 0.24997246,  0.        ,  0.5       ],
       [ 0.75002754,  0.        ,  0.5       ],
       [ 0.        ,  0.24998098,  0.        ],
       [ 0.        ,  0.75001902,  0.        ],
       [ 0.74997246,  0.5       ,  0.5       ],
       [ 0.25002754,  0.5       ,  0.5       ]])

In [173]:
spg.get_symmetry_dataset?

In [177]:
spg.get_spacegroup(cc,symprec=1e-4)

u'Cmmm (65)'

In [249]:
data = spg.get_symmetry_dataset(cc,symprec=1e-4)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
data.keys()
# data

65 ['i', 'i', 'h', 'h', 'h', 'h', 'i', 'i'] [0 0 2 2 2 2 0 0]


['std_positions',
 'equivalent_atoms',
 'std_types',
 'translations',
 'rotations',
 'number',
 'choice',
 'transformation_matrix',
 'origin_shift',
 'hall_number',
 'pointgroup',
 'international',
 'wyckoffs',
 'hall',
 'std_lattice']

In [263]:
sg,it = 65,1
cc = crystals[sg][it]
print cc.get_scaled_positions()[0]
print cc.get_cell_lengths_and_angles()
print isInSP(cc,sg)
symprec = get_symprec(cc,sg)
data = spg.get_symmetry_dataset(cc,symprec=symprec)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
print data['std_positions']
view(cc)

[ 0.12500016  0.06249997  0.87500515]
[   6.78590329    4.52128294    4.52096173   90.00000062   90.00000016
  109.45930452]
True
65 ['k', 'n', 'q', 'q', 'p', 'p', 'l', 'n', 'n', 'k', 'p', 'p', 'q', 'q', 'n', 'l'] [0 1 2 2 4 4 6 1 1 0 4 4 2 2 1 6]
[[ 0.          0.          0.74999485]
 [ 0.5         0.75000016  0.75000329]
 [ 0.25000119  0.624999    0.5       ]
 [ 0.24999881  0.875001    0.5       ]
 [ 0.75000091  0.62500133  0.        ]
 [ 0.74999909  0.87499867  0.        ]
 [ 0.5         0.          0.25000104]
 [ 0.          0.74999984  0.24999671]
 [ 0.5         0.75000016  0.24999671]
 [ 0.          0.          0.25000515]
 [ 0.25000091  0.87499867  0.        ]
 [ 0.24999909  0.62500133  0.        ]
 [ 0.75000119  0.875001    0.5       ]
 [ 0.74999881  0.624999    0.5       ]
 [ 0.          0.74999984  0.75000329]
 [ 0.5         0.          0.74999896]
 [ 0.5         0.5         0.74999485]
 [ 0.          0.25000016  0.75000329]
 [ 0.75000119  0.124999    0.5       ]
 [ 0.749998

In [91]:
spg.get_spacegroup(rr,symprec=)

u'Cmcm (63)'

In [193]:
ss = spg.get_symmetry_from_database?

In [213]:
ss = aspg.crystal(symbols=['Si'],basis=[0.3548618,0.3548618,0.1],cellpar=[8 ,   8,  3 , 90.  ,        90.    ,      90],spacegroup=35)
view(ss)

(0, 0, 0)+  (1/2, 1/2, 0)+  
(1) x, y, z	(2) -x, -y, z	(3) x, -y, z	(4) -x, y, z

In [207]:
sg,it = 35,1
cc = crystals[sg][it]
print cc.get_scaled_positions()

[[ 0.6451382   0.64752901  0.216     ]
 [ 0.3548618   0.35247099  0.216     ]
 [ 0.6451382   0.35247099  0.216     ]
 [ 0.3548618   0.64752901  0.216     ]
 [ 0.1451382   0.14752901  0.216     ]
 [ 0.8548618   0.85247099  0.216     ]
 [ 0.1451382   0.85247099  0.216     ]
 [ 0.8548618   0.14752901  0.216     ]]


In [82]:
print dd.get_cell_lengths_and_angles()
print dd.get_scaled_positions()[0]

[  3.12444082   7.72951641   3.1713511   90.          90.          90.        ]
[ 0.24999444  0.60755706  0.25000888]


# check wyckoff pos

In [5]:
def get_primitive(frame,sg):
    symprec = get_symprec(frame, sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(
                            frame, to_primitive=False,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)
    return primitive_atoms

In [6]:
highestMultiplicitySite = WyckTable[50].loc[0,:]
wl = highestMultiplicitySite['Wyckoff letter']
print wl

m


In [10]:
def match_with_ase(crystal,sg,tol=1e-5):
    Ncc = crystal.get_number_of_atoms()
    ee = aspg.crystal(symbols=['Si'],basis=crystal.get_scaled_positions()[0],cell=crystal.get_cell(),spacegroup=sg)
    Nee = ee.get_number_of_atoms()
    if Ncc == Nee:
        aa = np.allclose(ee.get_scaled_positions(),crystal.get_scaled_positions(),atol=tol)
        bb = np.allclose(ee.get_cell(),crystal.get_cell(),atol=tol)
        return aa and bb
    else:
        return False

In [55]:
# sg,it = not_unique[0]
sg,it = 224,0
print sg,it
crystal = crystals[sg][it]
ee = aspg.crystal(symbols=['Si'],basis=crystal.get_scaled_positions()[0],cell=crystal.get_cell(),spacegroup=sg)
print np.allclose(ee.get_scaled_positions(),crystal.get_scaled_positions(),atol=1e-5)
view(crystal)
view(ee)

224 0
True


In [53]:
sg,it = 228,2
print sg,it
crystal = crystals[sg][it]
print spg.get_spacegroup(crystal,symprec=1e-5)
print match_with_ase(crystal,sg,tol=1e-5)

228 2
Fd-3c (228)
True


In [51]:
try:
    missClassificationCorrection[sg]
except:
    print 'not in the list'
# symprec = get_symprec(crystal,sg)
spg.get_spacegroup(crystal,symprec=1e-2)

not in the list


u'Im-3m (229)'

In [33]:
symprec = get_symprec(crystal,sg)
data = spg.get_symmetry_dataset(crystal,symprec=symprec)
print data['number'],data['equivalent_atoms'],data['wyckoffs']

10 [0 0 0 0] ['o', 'o', 'o', 'o']


In [57]:
wrong_wl

[(12, 4),
 (51, 0),
 (51, 1),
 (51, 2),
 (51, 4),
 (65, 3),
 (65, 4),
 (72, 8),
 (83, 2),
 (123, 1),
 (123, 3),
 (123, 4),
 (127, 4),
 (174, 2),
 (175, 0),
 (191, 0),
 (215, 2),
 (217, 2),
 (224, 1),
 (227, 1)]

In [36]:
not_unique = []
miss = []
wrong_wl = []
good = []
tot = 0
for sg,cl in crystals.iteritems():
    highestMultiplicitySite = WyckTable[sg].loc[0,:]
    wl = highestMultiplicitySite['Wyckoff letter']
    for it,crystal in enumerate(cl):
        tot += 1
        symprec = get_symprec(crystal,sg)
        if symprec is None:
            print sg,it
            continue
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
#         print data['number'],data['wyckoffs'],data['equivalent_atoms']
        if sg in missClassificationCorrection.keys():
            miss.append((sg,it))
        elif len(np.unique(data['equivalent_atoms'])) > 1:
            not_unique.append((sg,it))
        elif data['wyckoffs'][0] != wl:
            wrong_wl.append((sg,it))
        else:
            good.append((sg,it))
Nu,Nm,Nw,Ng = len(not_unique),len(miss),len(wrong_wl),len(good)
print Nu,Nm,Nw,Ng,tot

7 0
7 2
7 3
8 4
11 1
11 3
12 1
13 0
13 1
13 2
13 6
14 0
15 1
15 2
15 4
15 5
15 6
15 7
15 8
15 9
15 14
16 0
16 1
16 2
16 3
16 4
17 0
17 1
17 2
17 3
17 4
18 0
18 1
18 2
18 3
18 4
19 0
19 1
19 2
19 4
20 0
20 1
20 2
20 3
20 4
20 5
20 7
20 8
20 9
20 10
20 11
20 12
20 13
21 0
21 1
21 2
21 3
21 4
21 5
21 6
21 8
21 9
22 0
22 3
23 0
23 1
23 2
23 3
23 4
24 0
24 1
24 2
24 3
24 4
26 0
26 3
27 1
27 3
27 4
28 4
29 2
29 3
30 0
30 1
30 3
31 0
31 4
31 6
33 1
33 3
34 1
34 4
35 4
36 1
36 6
37 0
37 1
37 2
37 7
38 0
38 4
39 0
39 2
39 3
39 4
40 0
40 1
40 4
40 6
41 1
41 3
41 7
42 1
43 3
43 4
43 5
44 0
45 0
45 1
45 2
45 3
48 1
48 2
48 3
48 4
48 5
48 7
50 0
50 1
50 2
50 3
50 4
52 1
52 2
52 5
52 6
52 7
52 8
52 9
52 10
52 11
52 12
53 0
53 1
53 4
54 0
54 2
54 3
54 4
55 0
55 1
55 2
56 2
56 4
56 5
57 0
57 1
57 2
57 4
57 5
57 6
57 7
57 8
58 0
58 1
58 3
58 4
59 0
59 2
59 3
59 4
60 0
60 1
60 3
60 4
61 0
61 1
61 2
61 4
62 1
62 2
62 3
62 4
62 5
62 6
62 8
62 9
63 1
63 2
64 0
64 2
64 3
64 4
65 0
65 1
65 2
66 0
66 1
66 3
6

In [273]:
print Nu,Nm,Nw,Ng,tot

68 283 66 797 1214


In [309]:
len(missClassificationCorrection)

53

### test pretty inspection

In [62]:
from ipywidgets import Layout, Button, Box,HTML
from bs4 import BeautifulSoup
sg = 4
# url of space group info at International Tables for Crystallography Volume A: Space-group symmetry
baseUrl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o'
url = baseUrl + '{:03.0f}'.format(sg)+'/'

# get the xml from the web page
soup = BeautifulSoup(requests.get(url).text, "lxml")

# get list of table of class genpos (-> general position part) and specpos (-> special positions)
genposList = soup.find_all("table", class_="genpos")
specposList = soup.find_all("table", class_="specpos")
genposcoords = soup.find_all("table", class_="genposcoords")
aa = genposList[0]
#     aa += specposList[1]
aa.append(specposList[0])

items = [
    HTML(aa.prettify())
 ]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='50%')
box = Box(children=items, layout=box_layout)
box



NameError: name 'BeautifulSoup' is not defined

In [64]:
def get_sg_table(sg):
    import requests
    from bs4 import BeautifulSoup
    from IPython.display import HTML

    # url of space group info at International Tables for Crystallography Volume A: Space-group symmetry
    baseUrl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o'
    url = baseUrl + '{:03.0f}'.format(sg)+'/'

    # get the xml from the web page
    soup = BeautifulSoup(requests.get(url).text, "lxml")

    # get list of table of class genpos (-> general position part) and specpos (-> special positions)
    genposList = soup.find_all("table", class_="genpos")
    specposList = soup.find_all("table", class_="specpos")
    aa = genposList[0]
    if len(specposList) > 0:
        aa.append(specposList[0])
    return aa

def print_sg_table(sg1,sg2):
    from ipywidgets import Layout, Button, Box,HTML
    from bs4 import BeautifulSoup
    t1 = get_sg_table(sg1)
    t2 = get_sg_table(sg2)
    items = [HTML(t1.prettify()),HTML(t2.prettify())]

    box_layout = Layout(display='flex',
                        flex_flow='row',
                        align_items='stretch',
                        border='solid',
                        width='100%')
    box = Box(children=items, layout=box_layout)
    return box
print_sg_table(10,12)

## Analysis

In [462]:
def isInSP(crystal, spaceGroupIdx, tols=[1e-5, 1e-4, 1e-3, 1e-2]):
    SPgood = False
    detectedSpaceGroup = []

    for tol in tols:
        detectedSpaceGroup.append(crystal2SpaceGroupIdx(crystal, tol))
    print detectedSpaceGroup
    if spaceGroupIdx in detectedSpaceGroup:
        SPgood = True
    elif spaceGroupIdx in missClassificationCorrection.keys():
        if missClassificationCorrection[spaceGroupIdx] in detectedSpaceGroup:
            SPgood = True
    return SPgood
def crystal2SpaceGroupIdx(crystal,symtol):
    '''Identify the space group index (from 1,230) of a given crystal (ase Atoms class).
        Return an int from 1 to 230.'''
    try:
        return int(spg.get_spacegroup(crystal,symprec=symtol).split(' ')[1][1:-1])
    except:

        return -1

In [485]:
for iii in range(11):
    isInSP(crystals[28][iii],28)

[51, 51, 65, 65]
[10, 10, 51, 51]
[10, 10, 51, 191]
[10, 10, 10, 51]
[10, 10, 51, 51]
[25, 51, 47, 47]
[51, 51, 63, 194]
[51, 51, 65, 191]
[10, 51, 51, 65]
[6, 51, 51, 51]
[10, 10, 51, 51]


In [500]:
for it in [16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 47, 48, 49, 50, 51,
                   52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]:
    print it,SGTable[SGTable['Table No.']==it]['Hall symbol'].values[0]

16 P 2 2
17 P 2c 2
18 P 2 2ab
19 P 2ac 2ab
25 P 2 -2
26 P 2c -2
27 P 2 -2c
28 P 2 -2a
29 P 2c -2ac
30 P 2 -2bc
31 P 2ac -2
32 P 2 -2ab
33 P 2c -2n
34 P 2 -2n
47 -P 2 2
48 P 2 2 -1n
49 -P 2 2c
50 P 2 2 -1ab
51 -P 2a 2a
52 -P 2a 2bc
53 -P 2ac 2
54 -P 2a 2ac
55 -P 2 2ab
56 -P 2ab 2ac
57 -P 2c 2b
58 -P 2 2n
59 P 2 2ab -1ab
60 -P 2n 2ab
61 -P 2ac 2ab
62 -P 2ac 2n


In [65]:
print_sg_table(196,202)

In [552]:
it = 0
aa = missClassificationCorrection.items()
print 'SG left = {}                                                              SG right = {}'.format(aa[it][0],aa[it][1])


sg = aa[it][0]

for ii,cc in enumerate(crystals[sg]):
    symprec = get_symprec(cc,sg)
    data = spg.get_symmetry_dataset(cc,symprec=symprec)
    
    print ii,data['wyckoffs'],data['number']
#     print data['std_positions']
    print cc.get_scaled_positions()
#     print cc.get_cell_lengths_and_angles()
print_sg_table(aa[it][0],aa[it][1])
# print_sg_table(aa[it][0],51)

SG left = 30                                                              SG right = 53
0 ['h', 'h', 'h', 'h'] 53
[[ 0.75001996  0.87499361  0.49      ]
 [ 0.24998004  0.12500639  0.49      ]
 [ 0.75001996  0.62500639  0.99      ]
 [ 0.24998004  0.37499361  0.99      ]]
1 ['h', 'h', 'h', 'h'] 53
[[ 0.2500044   0.37498106  0.556     ]
 [ 0.74999532  0.62501894  0.556     ]
 [ 0.25000468  0.12501894  0.056     ]
 [ 0.7499956   0.87498106  0.056     ]]
2 ['h', 'h', 'h', 'h'] 53
[[ 0.24999719  0.84244628  0.487     ]
 [ 0.75000281  0.15755372  0.487     ]
 [ 0.24999719  0.65755372  0.987     ]
 [ 0.75000281  0.34244628  0.987     ]]
3 ['h', 'h', 'h', 'h'] 53
[[  2.50044220e-01   2.81118353e-06   2.18999991e-01]
 [  7.49955780e-01   9.99997189e-01   2.19000009e-01]
 [  2.50044220e-01   4.99997286e-01   7.18999991e-01]
 [  7.49955780e-01   5.00002714e-01   7.19000009e-01]]
4 ['h', 'h', 'h', 'h'] 53
[[  2.49995007e-01   4.99996722e-01   2.89000002e-01]
 [  7.50004973e-01   5.00003278e-01   2.

In [551]:
sg,it = 29,2
cc = crystals[sg][it]
ee = aspg.crystal(symbols=['Si'],basis=cc.get_scaled_positions()[0],cellpar=cc.get_cell_lengths_and_angles(),spacegroup=sg)
print np.allclose(ee.get_scaled_positions(),cc.get_scaled_positions(),atol=1e-5)
print np.allclose(ee.get_cell(),cc.get_cell(),atol=1e-5)
view(cc)
view(ee)

False
True


In [290]:
wyck2match = {
    3:['m', 'n'],
    4:['e'],
    5:['i'],
    6:['i','j'],
    7:['e','f'],
    8:['g','h'],
    9:['e'],
    25:['z','y'],
    26:['j'],
    27:['q','i','j','k','l'],
    28:['j'],
    29:['d']
    
    
}

In [306]:
cc = crystals[sg][3]
dd = get_standard_frame(cc,sg)
print dd.get_scaled_positions()

[[ 0.75105019  0.5         0.60897755]
 [ 0.24894981  0.5         0.39102245]]


In [307]:
symprec=1e-5
data = spg.get_symmetry_dataset(cc,symprec=symprec)
print data['wyckoffs']
print spg.get_spacegroup(cc,symprec=symprec)
(lattice, positions, numbers) = spg.standardize_cell(
                            cc, to_primitive=False,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
dd = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)
print dd.get_scaled_positions()
print cc.get_cell()
print data['std_lattice']

['n', 'n']
P2/m (10)
[[ 0.75105019  0.5         0.60897755]
 [ 0.24894981  0.5         0.39102245]]
[[  2.25872516e+00   3.08179506e-19   2.38582043e-05]
 [  2.41545830e-18   2.23751551e+00   4.52021698e-19]
 [  4.49559691e+00   3.54006381e-18   9.04446916e+00]]
[[ 2.25872516  0.          0.        ]
 [ 0.          2.23751551  0.        ]
 [-0.02175788  0.          9.04442168]]
